In [1]:
pip install cvxpy-base


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 9.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import cvxpy as cp


In [ ]:
from IPython.display import *

# **Question 1**


# Supply Chain and Raw Ingredients Supply
<br>
    <ul>
        <li>A food factory requires $2000 tons of canola oil every month as a raw ingredient.</li>
        <br>
        <li>The prices of canola oil fluctuates from month to month due to market conditions.</li>
        <br>
        <li>The predicted prices for the next six months are: <br> <table>
  <tr>
    <th>Month</th>
    <th>1</th>
      <th>2</th>
      <th>3</th>
      <th>4</th>
      <th>5</th>
      <th>6</th>
  </tr>
  <tr>
    <td>Price per ton</td>
    <th>150</th>
      <th>160</th>
      <th>180</th>
      <th>170</th>
      <th>180</th>
      <th>160</th>
  </tr>
    </table></li>
        <br>
        <li>The factory's supplier for canola oil delivers it on the first day of every month, and charges the above prices.</li>
        <br>
        <li>The factory can decide how much oil to buy each month from the supplier.</li>
        <br>
        <li>At the end of each month, the factory can also store unused oil for future use, but the inventory of oil (total amount stored)  cannot exceed $1000 tons at any given time. Assume the inventory before the first shipment is zero.</li>
        <br>

In [ ]:
#Create a data frame featuring the necessary variables and information

food_production_pddf = pd.DataFrame(
    {
        "month": [1, 2, 3, 4, 5, 6],
        "price_per_ton": [150, 160, 180, 170, 180, 160],
        "demand": [2000, 2000, 2000, 2000, 2000, 2000],
        "max_storage": [1000, 1000, 1000, 1000, 1000, 1000]
    }
)

In [ ]:
food_production_pddf

,month,price_per_ton,demand,max_storage
0,1,150,2000,1000
1,2,160,2000,1000
2,3,180,2000,1000
3,4,170,2000,1000
4,5,180,2000,1000
5,6,160,2000,1000


In [ ]:
# Determine the variables that used in this optimization problem. Here we are tyring to determine how much oil to buy and how much to store
fp_dv_buy = cp.Variable(food_production_pddf.shape[0], nonneg = True)
fp_dv_store = cp.Variable(food_production_pddf.shape[0], nonneg = True)

In [ ]:
non_neg_const_fp_dv_buy, non_neg_const_fp_dv_store = (True, True)
fp_dv_buy, fp_dv_store = (cp.Variable(food_production_pddf.shape[0], nonneg = non_neg_const_fp_dv_buy), cp.Variable(food_production_pddf.shape[0], nonneg = non_neg_const_fp_dv_store))

In [ ]:
# Here we are determining the objective to this optimization which is to limit cost
total_procurement_cost = cp.sum(cp.multiply(food_production_pddf["price_per_ton"], fp_dv_buy))
obj_min_total_procurement_cost = cp.Minimize(total_procurement_cost)
obj_min_total_procurement_cost = cp.Minimize(cp.sum(cp.multiply(food_production_pddf["price_per_ton"], fp_dv_buy)))

In [ ]:
(fp_dv_store[i] == (fp_dv_buy[i] - food_production_pddf.loc[i, "demand"]) if i == 0 else
      (fp_dv_store[i] == (fp_dv_store[i - 1] + fp_dv_buy[i] - food_production_pddf.loc[i, "demand"]))
      for i in range(food_production_pddf.shape[0]))

<generator object <genexpr> at 0x28a6ce020>

In [ ]:
# Here we determine the constraints of this optimization problem which decided that we cannot store oil more than the max storage which is 1000 tons in this case
fp_constraints = []
for i in range(food_production_pddf.shape[0]):
  if i == 0:
    fp_constraints.append(fp_dv_store[i] == (fp_dv_buy[i] - food_production_pddf.loc[i, "demand"]))
  else:
     fp_constraints.append((fp_dv_store[i] == (fp_dv_store[i - 1] + fp_dv_buy[i] - food_production_pddf.loc[i, "demand"])))
fp_constraints.append(fp_dv_store <= food_production_pddf["max_storage"])

In [ ]:
fp_constraints = [
    *(fp_dv_store[i] == (fp_dv_buy[i] - food_production_pddf.loc[i, "demand"]) if i == 0 else
      (fp_dv_store[i] == (fp_dv_store[i - 1] + fp_dv_buy[i] - food_production_pddf.loc[i, "demand"]))
      for i in range(food_production_pddf.shape[0])),
    fp_dv_store <= food_production_pddf["max_storage"]
]

In [ ]:
fp_problem = cp.Problem(obj_min_total_procurement_cost, fp_constraints)

In [ ]:
# The solution is as follows:

optimal_cost = fp_problem.solve(verbose = True)

food_production_pddf["optimal_buys"] = fp_dv_buy.value
food_production_pddf["optimal_stores"] = fp_dv_store.value

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 11 12:48:24 PM: Your problem has 12 variables, 7 constraints, and 0 parameters.
(CVXPY) Jan 11 12:48:24 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 11 12:48:24 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 11 12:48:24 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 11 12:48:24 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 11 12:48:24 PM: Compiling problem (target solver=SCIPY).
(CV

In [ ]:
# Write down the solution in text.
# I also included a table detailing demand, maximum storage, optimal buys, and optimal storage for Canola Oil.

display(Markdown("<b><u>Based on the solver, we can define the problem as linear, therefore, we can fit them into a linear model, based on the fitted model, we can see the optimal buying quantity strategy for each month which are: </u></b>"),
        food_production_pddf)

<b><u>Based on the solver, we can define the problem as linear, therefore, we can fit them into a linear model, based on the fitted model, we can see the optimal buying quantity strategy for each month which are: </u></b>

,month,price_per_ton,demand,max_storage,optimal_buys,optimal_stores
0,1,150,2000,1000,3000.0,1000.0
1,2,160,2000,1000,2000.0,1000.0
2,3,180,2000,1000,1000.0,0.0
3,4,170,2000,1000,3000.0,1000.0
4,5,180,2000,1000,1000.0,0.0
5,6,160,2000,1000,2000.0,0.0


# **Question 2**

# Maximizing Manufacturing Profit / Revenue
<br>
    <ul>
        The Magnetron Company manufactures and markets microwave ovens.
        <br>
        <br>
        <ul>
            <li>Currently, the company produces two models: full-size and compact.</li>
            <br>
            <li>Production is limited by the amount of labor available in the general assembly and electronics assembly departments, as well as by the demand for each model.</li>
            <br>
            <li>Each full-size oven requires $2$ hours of general assembly and $2$ hours of electronic assembly, whereas each compact oven requires $2$ hours of general assembly and $3$ hours of electronic assembly.</li>
            <br>
            <li>In the current production period, there are $500$ hours of general assembly labor available and $800$ hours of electronic assembly labor available.</li>
            <br>
            <li>Additionally, the company estimates it can sell at most $220$ full-size ovens and $180$ compact ovens in the current production period.</li>
            <br>
            <li>The earnings contribution per oven is $120$ dollars for a full-size oven and $130$ dollars for a compact oven.</li>
            <br>
            <li>The company would like to find an earnings-maximizing production plan for the current production period.</li></font>
            <br>
        </ul>
</font>

In [ ]:
#Identify the variables outlined in the problem above. The problem specifies the production of two models (equivalent to two variables), for which the company seeks to devise a production plan aimed at maximizing revenue.
x = cp.Variable(1)
y = cp.Variable(1)

In [ ]:
#Determine the objective of this problem, which is to maximize production revenue
production_objective = (120*x + 130*y)
max_total_revenue = cp.Maximize(production_objective)

In [ ]:
#Identify the constraints as listed above
constraints  = [2*x + 2*y <= 500,
                2*x + 3*y <= 800,
                x <= 220, y <= 180,
                x >= 0, y >= 0]

In [ ]:
#Code the solution
problem = cp.Problem(max_total_revenue, constraints)
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 11 01:07:23 PM: Your problem has 2 variables, 6 constraints, and 0 parameters.
(CVXPY) Jan 11 01:07:23 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 11 01:07:23 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 11 01:07:23 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 11 01:07:23 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 11 01:07:23 PM: Compiling problem (target solver=SCIPY).
(CVX

31800.0

In [ ]:
# This text includes the amount of units that can be produced to generate a maximum profit with the most efficient labor cost as detailed below:

display(Markdown("<b><u>The production plan is to produce 70 units of the full-size ovens and 180 units of the compact size oven, which gives a maximum earning of $31,800 dollars per this production period as identified by the solver.</u></b>"))

<b><u>The production plan is to produce 70 units of the full-size ovens and 180 units of the compact size oven, which gives a maximum earning of $31,800 dollars per this production period as identified by the solver.</u></b>

# **Question 3**

# Supply Chain Planning
<font size="+1">
    <ul>
        <br>
        <li>Every time Amazon has to ship an item from its fulfillment centers to a particular region, it incurs a shipping cost.</li>
        <br>
        <li>Consider the following shipping costs from three of Amazon's fulfillment centers (FC) to four regions (A, B, C, and D):</li>
        <br>
        <br><table>
  <tr>
    <th>Region / FC:</th>
    <th>1</th>
    <th>2</th>
      <th>3</th>
  </tr>
  <tr>
    <td>A: King's County, NY</td>
    <td>20</td>
    <td>8</td>
      <td>25</td>
  </tr>
            <tr>
    <td>B: LA County, CA</td>
    <td>18</td>
      <td>23</td>
      <td>8</td>
  </tr>
  <tr>
    <td>C: King County, WA</td>
    <td>21</td>
    <td>24</td>
      <td>8</td>
  </tr>
            <tr>
    <td>D: Harris County, TX</td>
    <td>8</td>
    <td>8</td>
      <td>19</td>
  </tr>
       </table>
        <br>
        <li>It costs $20 dollars to ship from fulfillment center 1 to region A, for example.</li>
        <br>
        <li>We also have weekly demand estimates for the item in each region:</li>
        <br>
        <table>
  <tr>
    <th>Region A</th>
    <th>Region B</th>
    <th>Region C</th>
      <th>Region D</th>
  </tr>
             <tr>
    <th>30</th>
    <th>50</th>
    <th>10</th>
      <th>20</th>
            </tr></table></li>
        <br>
        <li>Further, assume each fulfillment center is able to ship up to $40 units each week in total.</li>
        <br>
        <li>What is the minimum transportation cost needed to satisfy all weekly demand while respecting fulfillment center capacities, as well as the optimal shipment plan?</li>
        <br>
    </ul>
</font>

In [ ]:
# Determine the variables for this problem
fullfilment_centers = 3
regions = 4

x = cp.Variable((fullfilment_centers, regions))

In [ ]:
# Create a data frame / table detailing transportation costs

transportation_cost = pd.DataFrame(
    {"Region\FC" : ["A: King's County, NY", "B: LA County, CA", "C: King County", "D: Harris County, TX"],
     "1" : [20, 18, 21, 8],
     "2" : [8, 23, 24, 8],
     "3" : [25, 8, 8, 19]
    }
)
transportation_cost

,Region\FC,1,2,3
0,"A: King's County, NY",20,8,25
1,"B: LA County, CA",18,23,8
2,C: King County,21,24,8
3,"D: Harris County, TX",8,8,19


In [ ]:
transportation_cost_per_location = transportation_cost.iloc[0:4, 1:4].values.transpose()

In [ ]:
total_transportation_cost = sum([transportation_cost_per_location[a,b]*x[a,b] for a in range(x.shape[0]) for b in range(x.shape[1])])

In [ ]:
objective = cp.Minimize(total_transportation_cost)

In [ ]:
capacity_constraints = [(sum([x[a,b] for b in range(x.shape[1])]) <= 40) for a in range(x.shape[0])]

In [ ]:
region_demand = [30, 50, 10, 20]

demand_constraints = [(sum([x[a,b] for a in range(x.shape[0])]) >= region_demand[b]) for b in range(x.shape[1])]

In [ ]:
constraints = capacity_constraints + demand_constraints + [x>=0]

In [ ]:
problem = cp.Problem(objective, constraints)

In [ ]:
print(problem)

minimize 20.0 @ var829[0, 0] + 18.0 @ var829[0, 1] + 21.0 @ var829[0, 2] + 8.0 @ var829[0, 3] + 8.0 @ var829[1, 0] + 23.0 @ var829[1, 1] + 24.0 @ var829[1, 2] + 8.0 @ var829[1, 3] + 25.0 @ var829[2, 0] + 8.0 @ var829[2, 1] + 8.0 @ var829[2, 2] + 19.0 @ var829[2, 3]
subject to var829[0, 0] + var829[0, 1] + var829[0, 2] + var829[0, 3] <= 40.0
           var829[1, 0] + var829[1, 1] + var829[1, 2] + var829[1, 3] <= 40.0
           var829[2, 0] + var829[2, 1] + var829[2, 2] + var829[2, 3] <= 40.0
           30.0 <= var829[0, 0] + var829[1, 0] + var829[2, 0]
           50.0 <= var829[0, 1] + var829[1, 1] + var829[2, 1]
           10.0 <= var829[0, 2] + var829[1, 2] + var829[2, 2]
           20.0 <= var829[0, 3] + var829[1, 3] + var829[2, 3]
           0.0 <= var829


In [ ]:
# Use a solver to figure out the optimal shipment plan that reduces / minimizes cost
optimal_cost = problem.solve(verbose=True)
problem.value

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 11 01:14:32 PM: Your problem has 12 variables, 8 constraints, and 0 parameters.
(CVXPY) Jan 11 01:14:32 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 11 01:14:32 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 11 01:14:32 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 11 01:14:32 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 11 01:14:32 PM: Using cached ASA map, for faster compilation

1080.0

In [ ]:
# Text detailing the optimal shipment plan and the lowest shipping cost per week.
display(Markdown("<b><u>This tells us the optimal shipment plan from fulfillment centers to regions that minimizes shipping costs to $1080 per week.</u></b>"))

<b><u>This tells us the optimal shipment plan from fulfillment centers to regions that minimizes shipping costs to $1080 dollars per week.</u></b>